# Deep-Learning-from-Scratch
## Chap5. Backpropagation
## 5.4 단순한 계층 구현하기

다음 절에서는 '사과 쇼핑의 예'를 파이썬으로 구현한다..
* 계산 그래프의 곱셈 노드를 'MulLayer'
* 덧셈 노드를 'AddLayer'라는 이름으로 구현한다.

신경망을 구성하는 '계층'을 각각 하나의 클래스로 구현한다.<br/>
여기서 말하는 계층이란 신경망의 기능 단위이다. <br/>
예를 들어 시그모이드 함수를 위한 Sigmoid, 행렬 곱을 위한 Affine 등의 기능을 계층 단위로 구현한다. <br/>
이번 절에서도 곱셈 노드와 덧셈 노드를 '계층'단위로 구현한다.

### 5.4.1 곱셈 계층

모든 계층은 forward()와 backward()라는 공통의 메서드(인터페이스)를 갖도록 구현할 것이다. <br/>
forward()는 순전파, backward()는 역전파를 저치한다. <br/>
우선 곱셈 계층을 구현해보자. 곱셈 계층은 MultiLayer라는 이름의 클래스로 다음과 같이 구현할 수 있다.

In [4]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out

    def backward(self, dout):  # diyt: 상류에서 넘어온 미분값
                               # 상류에서 넘어온 미분에 순전파 때의 값을 '서로 바꿔'곱한 후 하류로 흘린다.
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

MultiLayer 클래스를 통해 사과 쇼핑을 구현해본다

In [6]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)  # 220

220.00000000000003


각 변수에 대한 미분은 backward()에서 구할 수 있다.

In [14]:
# 역전파

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_price, dtax, dapple, dapple_num)

1.1 200 2.2 110.00000000000001


* backward()가 받는 인수는 '순전파의 출력에 대한 미분'임에 주의하자.

### 5.4.2 덧셈 계층

In [15]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

* 사과 2개 구입 귤 3개 구입 구현

In [16]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 1. 계층들
# Node를 만들어준다.

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer=  MulLayer()

# 2. 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 3. 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

# 4. 출력
print(price)
print(dall_price, dapple_price, dorange_price, dorange, dorange_num, dapple, dapple_num)

715.0000000000001
1.1 1.1 1.1 3.3000000000000003 165.0 2.2 110.00000000000001
